# Investigate  energies of LJ clusters with mixed params

2024-05-01

Process:
- define LJ clusters where ca half of atoms has energy $\epsilon$ and other half eg $\epsilon/2$, or some general $\alpha\epsilon$
- minimise using basin hopping, store energies somewhere
- compute energy per atom and plot vs atom number

In [50]:
import numpy as np
import pandas as pd
from scipy.optimize import basinhopping
from datetime import datetime

import plotly.express as px

In [28]:
def potential(r, epsilon, sigma=1.0):
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

def total_energy(r, T, eps_dict, sigma=1.0):
    """Flat matrix of positions, `R.flatten()`"""
    V = 0.0
    R = r.reshape(-1, 3)
    for i, _ in enumerate(R):
        for j in range(i):
            rij = np.linalg.norm(R[i] - R[j])
            # V += potential(rij, eps_dict[(T[i], T[j])], sigma)
            V += potential(rij, 1.0, sigma)
    return V

In [29]:
def print_fun(x, f, accepted):
    print("at minimum %.4f accepted %d" % (f, int(accepted)))

## Trial simulations

In [46]:
alpha = 0.5
beta = 1.0

eps_dict = {
    (0, 0): 1.0,
    (1, 1): beta,
    (0, 1): alpha,
    (1, 0): alpha,
}

In [42]:
Z, N = 4, 4
A = Z + N

T = np.array([1] * Z + [0] * N)

np.random.seed(43)
R = np.random.uniform(0, 2, size=(A, 3)) - 1.0

In [43]:
total_energy(R.flatten(), T, eps_dict)

6027.809791057789

In [48]:
%%time

res = basinhopping(
    total_energy,
    R.flatten(),
    niter=100,
    minimizer_kwargs=dict(method='L-BFGS-B', args=(T, eps_dict, 1.0)),
    callback=print_fun,
    niter_success=10
)

at minimum -14.1197 accepted 1
at minimum -15.0042 accepted 1
at minimum -15.0042 accepted 1
at minimum -15.0042 accepted 1
at minimum -14.9649 accepted 1
at minimum -15.3263 accepted 1
at minimum -15.0042 accepted 1
at minimum -15.3263 accepted 1
at minimum -15.3865 accepted 1
at minimum -15.3865 accepted 1
at minimum -15.4621 accepted 1
at minimum -15.0428 accepted 0
at minimum -14.9737 accepted 0
at minimum -15.4621 accepted 1
at minimum -15.4621 accepted 1
at minimum -15.0428 accepted 1
at minimum -15.4621 accepted 1
at minimum -14.5215 accepted 1
at minimum -15.0042 accepted 1
at minimum -15.0042 accepted 1
at minimum -15.3263 accepted 1
at minimum -14.9649 accepted 1
at minimum -14.9649 accepted 1
at minimum -15.0042 accepted 1
at minimum -14.7900 accepted 1
at minimum -14.7900 accepted 1
CPU times: user 7.79 s, sys: 297 ms, total: 8.09 s
Wall time: 9.11 s


In [49]:
res.fun

-15.462145744307493

## Full minimisation of several atom numbers

In [47]:
alpha = 0.5
beta = 1.0

eps_dict = {
    (0, 0): 1.0,
    (1, 1): beta,
    (0, 1): alpha,
    (1, 0): alpha,
}

In [53]:
%%time

# n_vec = [1, 2, 3, 4, 5, 6]
n_vec = [7, 8, 9, 10, 15, 20] # , 30, 50, 70, 80, 90, 100]

for ni in n_vec:
    # define configuration
    Z, N = ni, ni
    A = Z + N

    T = np.array([1] * Z + [0] * N)

    np.random.seed(43)
    R = np.random.uniform(0, 2, size=(A, 3)) - 1.0

    # minimize
    print(f'Running {Z}, {N} configuration')
    ti = datetime.now()
    res = basinhopping(
        total_energy,
        R.flatten(),
        niter=100,
        minimizer_kwargs=dict(method='L-BFGS-B', args=(T, eps_dict, 1.0)),
        niter_success=max(10, ni * 2)
    )
    tf = datetime.now()

    print(f'Z: {Z}, N: {N}, Energy: {res.fun}, Energy per atom: {res.fun/(Z+N)}, Time: {tf - ti}')

Running 7, 7 configuration
Z: 7, N: 7, Energy: -37.863739259752116, Energy per atom: -2.7045528042680083, Time: 0:00:53.947273
Running 8, 8 configuration
Z: 8, N: 8, Energy: -46.14091946370434, Energy per atom: -2.8838074664815214, Time: 0:01:36.499190
Running 9, 9 configuration
Z: 9, N: 9, Energy: -56.80072335532324, Energy per atom: -3.1555957419624026, Time: 0:03:07.099256
Running 10, 10 configuration
Z: 10, N: 10, Energy: -65.87778036358849, Energy per atom: -3.2938890181794243, Time: 0:09:09.196979
Running 15, 15 configuration
Z: 15, N: 15, Energy: -111.4100198513263, Energy per atom: -3.7136673283775434, Time: 0:45:30.499135
Running 20, 20 configuration


KeyboardInterrupt: 

### Results

```
Running 1, 1 configuration
Z: 1, N: 1, Energy: -0.5, Time: 0:00:00.080495
Running 2, 2 configuration
Z: 2, N: 2, Energy: -3.9999999999999227, Time: 0:00:00.693397
Running 3, 3 configuration
Z: 3, N: 3, Energy: -9.354907562246039, Time: 0:00:02.249838
Running 4, 4 configuration
Z: 4, N: 4, Energy: -15.583345013386326, Time: 0:00:09.379156
Running 5, 5 configuration
Z: 5, N: 5, Energy: -22.851442945760887, Time: 0:00:15.601218
Running 6, 6 configuration
Z: 6, N: 6, Energy: -28.39335425926725, Time: 0:00:20.240499
Running 7, 7 configuration
Z: 7, N: 7, Energy: -37.863739259752116, Energy per atom: -2.7045528042680083, Time: 0:00:53.947273
Running 8, 8 configuration
Z: 8, N: 8, Energy: -46.14091946370434, Energy per atom: -2.8838074664815214, Time: 0:01:36.499190
Running 9, 9 configuration
Z: 9, N: 9, Energy: -56.80072335532324, Energy per atom: -3.1555957419624026, Time: 0:03:07.099256
Running 10, 10 configuration
Z: 10, N: 10, Energy: -65.87778036358849, Energy per atom: -3.2938890181794243, Time: 0:09:09.196979
Running 15, 15 configuration
Z: 15, N: 15, Energy: -111.4100198513263, Energy per atom: -3.7136673283775434, Time: 0:45:30.499135
```

In [54]:
# process results

s = """Running 1, 1 configuration
Z: 1, N: 1, Energy: -0.5, Time: 0:00:00.080495
Running 2, 2 configuration
Z: 2, N: 2, Energy: -3.9999999999999227, Time: 0:00:00.693397
Running 3, 3 configuration
Z: 3, N: 3, Energy: -9.354907562246039, Time: 0:00:02.249838
Running 4, 4 configuration
Z: 4, N: 4, Energy: -15.583345013386326, Time: 0:00:09.379156
Running 5, 5 configuration
Z: 5, N: 5, Energy: -22.851442945760887, Time: 0:00:15.601218
Running 6, 6 configuration
Z: 6, N: 6, Energy: -28.39335425926725, Time: 0:00:20.240499
Running 7, 7 configuration
Z: 7, N: 7, Energy: -37.863739259752116, Energy per atom: -2.7045528042680083, Time: 0:00:53.947273
Running 8, 8 configuration
Z: 8, N: 8, Energy: -46.14091946370434, Energy per atom: -2.8838074664815214, Time: 0:01:36.499190
Running 9, 9 configuration
Z: 9, N: 9, Energy: -56.80072335532324, Energy per atom: -3.1555957419624026, Time: 0:03:07.099256
Running 10, 10 configuration
Z: 10, N: 10, Energy: -65.87778036358849, Energy per atom: -3.2938890181794243, Time: 0:09:09.196979
Running 15, 15 configuration
Z: 15, N: 15, Energy: -111.4100198513263, Energy per atom: -3.7136673283775434, Time: 0:45:30.499135"""

In [55]:
s = [l for l in s.split('\n') if l[0] == 'Z']

In [115]:
d = [{r.split(':')[0].strip(): r.split(': ')[1].strip() for r in l.split(',')} for l in s]
dff = pd.DataFrame(d)
dff = dff.astype({'Z': int, 'N': int, 'Energy': float})
dff['Time'] = dff['Time'].apply(lambda x: pd.to_timedelta(x).total_seconds())

In [116]:
dff

,Z,N,Energy,Time,Energy per atom
0,1,1,-0.500000,0.080495,NaN
1,2,2,-4.000000,0.693397,NaN
2,3,3,-9.354908,2.249838,NaN
3,4,4,-15.583345,9.379156,NaN
4,5,5,-22.851443,15.601218,NaN
5,6,6,-28.393354,20.240499,NaN
6,7,7,-37.863739,53.947273,-2.7045528042680083
7,8,8,-46.140919,96.499190,-2.8838074664815214
8,9,9,-56.800723,187.099256,-3.1555957419624026
9,10,10,-65.877780,549.196979,-3.2938890181794243


In [117]:
dff['A'] = dff['Z'] + dff['N']
dff['ea'] = dff['Energy'] / (dff['Z'] + dff['N'])

In [118]:
px.scatter(dff, x='N', y='ea', title='Energy per atom vs N')

In [119]:
px.scatter(dff, x='N', y='Time', title='Time vs A', log_x=True, log_y=True)

In [124]:
np.polyfit(np.log(dff['A']), np.log(dff['Time']), 1)[0]

3.798161925073758